In [1]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import argparse
import pdb
import HEI
import sys
import networkx as nx

In [2]:
basepath='/Users/gracer/Google Drive/BCP/data/derivived'

In [3]:
convers = pd.read_csv('/Users/gracer/Google Drive/BCP/data/conversions.csv', sep=',')

In [18]:
convers['ID']=[x.split('_')[0] for x in convers['MotherPSCID'] ]

In [19]:
convers

,FamilyID,Relationship_type,Participant ID_x,ChildPSCID,Participant ID,MotherPSCID,ID
0,2,mother,627391,MNBCP000094_04,326619,MNBCP000094_01,MNBCP000094
1,8,mother,805063,JE000425_04,322454,JE000425_02,JE000425
2,9,mother,519250,MNBCP000073_04,45261,MNBCP000073_02,MNBCP000073
3,10,mother,522586,MNBCP000085_04,706544,MNBCP000085_02,MNBCP000085
4,11,mother,116845,JE000236_04,167383,JE000236_02,JE000236
5,12,mother,978698,JE000509_03,457901,JE000509_02,JE000509
6,13,mother,423548,JE000238_03,281070,JE000238_02,JE000238
7,15,mother,981664,MNBCP000060_05,957439,MNBCP000060_02,MNBCP000060
8,16,mother,258397,MNBCP000042_06,957492,MNBCP000042_02,MNBCP000042
9,17,mother,116056,JE000445_03,119470,JE000445_02,JE000445


In [25]:
mom_interest=['ID','Participant ID', 'Participant ID_x','Date of Intake','HEIX1_TOTALVEG','HEIX2_GREEN_AND_BEAN','HEIX3_TOTALFRUIT','HEIX4_WHOLEFRUIT','HEIX5_WHOLEGRAIN','HEIX6_TOTALDAIRY','HEIX7_TOTPROT','HEIX8_SEAPLANT_PROT','HEIX11_REFINEDGRAIN','HEIX12_ADDEDSUGARS','HEIX13_SATFATS','HEIX9_FATTYACID','HEIX10_SODIUM','HEI2015_TOTAL_SCORE', 'weight','Gender_y']
ped_interest=['ID','Participant ID_x','Participant ID', 'Date of Intake', 'Age at Intake','HEIX0_BREASTFEEDING','HEIX1_VEGETABLES','HEI2015_TOTAL_SCORE','HEIX2_TOTALFRUIT','HEIX3_WHOLEGRAIN','HEIX4_TOTALDAIRY','HEIX5_PROTEIN','HEIX6_REFINEDGRAIN','HEIX7_FRUITJUICE','HEIX8_SSB','HEIX9_SWEETS','HEIX10_SALTY', 'weight','Gender_y']
infant_interest=['ID','Participant ID_x','Participant ID', 'Date of Intake', 'Age at Intake','HEIX0_BREASTFEEDING','HEIX1_VEGETABLES','HEI2015_TOTAL_SCORE','HEIX2_TOTALFRUIT','HEIX3_WHOLEGRAIN','HEIX4_TOTALDAIRY','HEIX5_PROTEIN','HEIX6_REFINEDGRAIN','HEIX7_FRUITJUICE','HEIX8_SSB','HEIX9_SWEETS','HEIX10_SALTY', 'weight','Gender_y']

In [26]:
data_dict={'infant':{}, 'young':{}, 'child':{}, 'mom':{}}
for item in glob.glob(os.path.join(basepath,'*HEI.csv')):
    if 'infant' in item:
        key='infant'
        tmp=pd.read_csv(item, sep=',')
        tmp=pd.merge(tmp,convers, on=['Participant ID_x'])
        print('pre %s %s'%(key, tmp.shape))
        tmp.drop_duplicates(subset=['Participant ID_x', 'Date of Intake'], inplace=True)
        print('post %s %s'%(key, tmp.shape))
        tmp=tmp[infant_interest]
        tmp['time'] = 0
        data_dict['infant']=tmp
    if 'young' in item:
        key='young'
        tmp=pd.read_csv(item, sep=',')
        tmp=pd.merge(tmp,convers, on=['Participant ID_x'])
        print('pre %s %s'%(key, tmp.shape))
        tmp.drop_duplicates(subset=['Participant ID_x', 'Date of Intake'], inplace=True)
        print('post %s %s'%(key, tmp.shape))
        tmp=tmp[ped_interest]
        tmp['time'] = 1
        data_dict['young']=tmp
    if 'child' in item:
        key='child'
        tmp=pd.read_csv(item, sep=',')
        tmp=pd.merge(tmp,convers, on=['Participant ID_x'])
        print('pre %s %s'%(key, tmp.shape))
        tmp.drop_duplicates(subset=['Participant ID_x', 'Date of Intake'], inplace=True)
        print('post %s %s'%(key, tmp.shape))
        tmp=tmp[ped_interest]
        tmp['time'] = 2
        data_dict['child']=tmp
    if 'Mom' in item:
        key='Mom'
        tmp=pd.read_csv(item, sep=',')
        tmp=pd.merge(tmp,convers, on=['Participant ID'])
        print('pre %s %s'%(key, tmp.shape))
        tmp.drop_duplicates(subset=['Participant ID', 'Date of Intake'], inplace=True)
        print('post %s %s'%(key, tmp.shape))
        tmp=tmp[mom_interest]
        tmp['time'] = 'mom'
        data_dict['mom']=tmp

pre child (558, 249)
post child (209, 249)
pre infant (157, 250)
post infant (127, 250)
pre Mom (112, 415)
post Mom (87, 415)
pre young (126, 249)
post young (95, 249)


In [28]:
data_dict['infant']

,ID,Participant ID_x,Participant ID,Date of Intake,Age at Intake,HEIX0_BREASTFEEDING,HEIX1_VEGETABLES,HEI2015_TOTAL_SCORE,HEIX2_TOTALFRUIT,HEIX3_WHOLEGRAIN,HEIX4_TOTALDAIRY,HEIX5_PROTEIN,HEIX6_REFINEDGRAIN,HEIX7_FRUITJUICE,HEIX8_SSB,HEIX9_SWEETS,HEIX10_SALTY,time
0,JE000478,105040,1995,2017-03-01,6.275283,NaN,5,35,0,5,5,5,5,5,5,5,5,0
1,NCBCP0018,107008,648900,2017-04-22,7.425204,NaN,0,30,0,5,0,5,0,5,5,5,5,0
2,JE000445,116056,119470,2016-09-29,0.952792,NaN,0,25,0,0,0,5,0,5,5,0,5,0
3,JE000445,116056,119470,2016-12-18,3.581182,NaN,5,40,5,5,5,5,5,5,5,5,5,0
4,JE000445,116056,119470,2017-03-24,6.735251,NaN,0,25,0,0,0,5,0,5,5,5,5,0
5,JE000236,116845,167383,2016-01-06,6.603832,NaN,0,35,5,0,5,5,5,5,5,5,5,0
6,JE000236,116845,167383,2015-09-29,3.351198,NaN,5,40,5,5,5,5,5,5,5,5,5,0
7,JE000265,125632,838504,2016-03-31,4.698248,NaN,0,35,5,0,5,5,5,5,5,5,5,0
8,JE000265,125632,838504,2016-06-10,7.030945,NaN,0,30,0,0,5,5,0,5,5,5,5,0
9,JE000235,131015,317472,2015-10-12,5.223927,NaN,5,40,5,5,5,5,5,5,5,5,5,0


In [7]:
inf_young = list(set(data_dict['infant']['Participant ID_x']).intersection(data_dict['young']['Participant ID_x']))
len(inf_young)

34

In [8]:
young_child = list(set(data_dict['child']['Participant ID_x']).intersection(data_dict['young']['Participant ID_x']))
len(young_child)

41

In [9]:
inf2child = list(set(inf_young).intersection(young_child))
len(inf2child)

25

In [10]:
inf_child = list(set(data_dict['child']['Participant ID_x']).intersection(data_dict['infant']['Participant ID_x']))
len(inf_child)

41

In [11]:
inf_mom = list(set(data_dict['infant']['Participant ID']).intersection(data_dict['mom']['Participant ID']))
len(inf_mom)

32

In [12]:
young_mom = list(set(data_dict['young']['Participant ID']).intersection(data_dict['mom']['Participant ID']))
len(young_mom)

30

In [13]:
child_mom = list(set(data_dict['mom']['Participant ID']).intersection(data_dict['child']['Participant ID']))
len(child_mom)

35

In [29]:
kids=pd.concat([data_dict['infant'], data_dict['young'], data_dict['child']])

In [30]:
kids

,ID,Participant ID_x,Participant ID,Date of Intake,Age at Intake,HEIX0_BREASTFEEDING,HEIX1_VEGETABLES,HEI2015_TOTAL_SCORE,HEIX2_TOTALFRUIT,HEIX3_WHOLEGRAIN,HEIX4_TOTALDAIRY,HEIX5_PROTEIN,HEIX6_REFINEDGRAIN,HEIX7_FRUITJUICE,HEIX8_SSB,HEIX9_SWEETS,HEIX10_SALTY,time
0,JE000478,105040,1995,2017-03-01,6.275283,NaN,5.0,35.0,0.0,5.0,5,5,5.0,5.0,5.0,5.0,5.0,0
1,NCBCP0018,107008,648900,2017-04-22,7.425204,NaN,0.0,30.0,0.0,5.0,0,5,0.0,5.0,5.0,5.0,5.0,0
2,JE000445,116056,119470,2016-09-29,0.952792,NaN,0.0,25.0,0.0,0.0,0,5,0.0,5.0,5.0,0.0,5.0,0
3,JE000445,116056,119470,2016-12-18,3.581182,NaN,5.0,40.0,5.0,5.0,5,5,5.0,5.0,5.0,5.0,5.0,0
4,JE000445,116056,119470,2017-03-24,6.735251,NaN,0.0,25.0,0.0,0.0,0,5,0.0,5.0,5.0,5.0,5.0,0
5,JE000236,116845,167383,2016-01-06,6.603832,NaN,0.0,35.0,5.0,0.0,5,5,5.0,5.0,5.0,5.0,5.0,0
6,JE000236,116845,167383,2015-09-29,3.351198,NaN,5.0,40.0,5.0,5.0,5,5,5.0,5.0,5.0,5.0,5.0,0
7,JE000265,125632,838504,2016-03-31,4.698248,NaN,0.0,35.0,5.0,0.0,5,5,5.0,5.0,5.0,5.0,5.0,0
8,JE000265,125632,838504,2016-06-10,7.030945,NaN,0.0,30.0,0.0,0.0,5,5,0.0,5.0,5.0,5.0,5.0,0
9,JE000235,131015,317472,2015-10-12,5.223927,NaN,5.0,40.0,5.0,5.0,5,5,5.0,5.0,5.0,5.0,5.0,0
